__Purpose__:
"I would approach that by training a "current best meta model" on most/all of the extant transcriptions aggregated together, then apply that to a couple of the more complete transcriptions and graph the check results to see how everything is working"

## Notes

"load_model()" with no params fed in does not load a model, it creates a new one

Hyper parameters from NB 70
1. Dropout of 0.5
2. Learning rate of 0.0025 (0.001?)

In [ ]:
#no_test

#data structure imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from os import listdir

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *
from ssda_nlp.utility import *

#if you want the same model everytime:
seed = 2436
random.seed(seed)
fix_random_seed(seed)

## Train Various Models
1. Train a number of models with different training data

In [ ]:
#no_test
#OUTPUT MUST BE COLLATED_DF

#These file paths are from the Github repo, should work on anyone's computer
cdf1 = prodigy_output_to_collated_df("transcriptions/15834_annot.jsonl")
cdf2 = prodigy_output_to_collated_df("transcriptions/239746_annot.jsonl")
cdf3 = prodigy_output_to_collated_df("transcriptions/419324_annot.jsonl")
cdf4 = prodigy_output_to_collated_df("transcriptions/701075_annot.jsonl")
cdf5 = prodigy_output_to_collated_df("transcriptions/guatemala.annotations.jsonl")

full_collated_df = cdf1.append(cdf2).append(cdf3).append(cdf4).append(cdf5)
assert full_collated_df.shape[0] == (cdf1.shape[0] + cdf2.shape[0] + cdf3.shape[0] + cdf4.shape[0] + cdf5.shape[0])

full_collated_df = full_collated_df.sample(frac = 1)
cdf80 = full_collated_df.sample(frac = 0.8)

my_cdfs = [cdf1, cdf2, cdf3, cdf4, cdf5, cdf80]

In [ ]:
#no_test

#for training_data in training_log_LanCDS2:
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.5

for idx, cdf in enumerate(my_cdfs):
    # Already trained them so just ignore this unless you want to retrain
    """
    save_mdir = "models/75/" + str(idx)
    
    train_df, valid_df, test_df = split_data_grp(cdf, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)
    # Generate Spacy datasets
    training_data = genSpaCyInput(train_df)

    #load model from pretrained
    nlp_model = load_model()
    nlp_model, perf_df = model_meta_training(nlp_model, training_data, valid_df, verbose=True, save_dir = save_mdir,
                                             n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)
    display(perf_df.head(10))
    """

display('Successful run')

'Successful run'

## Test The Above Models
1. Determine the best meta-model: winner take all
- test_model(nlp_model, testing_df, id_colname, text_colname, score_model=True)

In [4]:
full_collated_df.head()

,entry_no,text,entity,start,end,label
855,64,[margin]: Simiana. Aos doze dias do mes de Jan...,Joam Pimenta,115,127,PER
1376,108,[margin]: Joam. Em o primeiro dia de junho de ...,Joam,10,14,PER
2370,175,[margin]: Natalia. Ao primeiro dia do mes de j...,primeiro dia do mes de janeiro de 1696,22,60,DATE
1906,148,[margin]: Agueda. Aos treze dias do mes de jul...,Faustino Correa,141,156,PER
333,12,"Eusebio Joseph Ramon Guadara Martes, día cinco...",padrinos,604,612,REL


In [11]:
#no_test

test_df = full_collated_df
path = 'models/75'

for my_model in listdir(path):
    my_model = path + r"/" + my_model
    
    trained_model = load_model(my_model, language="es", verbose='True')

    ent_preds_df, metrics_df, per_ent_metrics = test_model(trained_model, test_df, 'entry_no', 'text')

    #show performance
    print('\nOverall performance:')
    display(metrics_df)
    print('\nEntity-specific performance:')
    display(per_ent_metrics)
    print("-----------------------------------------------------")

Loaded model 'models/75/0'


C:\Users\kdmen\miniconda3\envs\ssda\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (234, 258, 'PER'), (221, 225, 'C...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,



Overall performance:


,precision,recall,f_score
0,52.682345,44.843173,48.4477



Entity-specific performance:


...


Overall performance:


,precision,recall,f_score
0,92.216777,88.53321,90.337459



Entity-specific performance:


,DATE,PER,REL,LOC,CHAR,PERSON,LOCATION,RELATIONSHIPS,CHARACTERISTICS
p,0.884434,0.929605,0.945046,0.933182,0.902439,0.931280,0.954545,0.950450,0.984375
r,0.833333,0.880986,0.934926,0.913525,0.877037,0.825630,0.954545,0.981395,0.969231
f,0.858124,0.904643,0.939959,0.923249,0.889557,0.875278,0.954545,0.965675,0.976744


-----------------------------------------------------


In [18]:
# no_test

# Hard coded: the best meta model, as seen in the tests above, is:
# Note that this only works because the last model was the best model
best_meta_model = trained_model
save_dir = path + r"/" + "best_meta_model"

save_model(best_meta_model, save_dir)

Saved model to models\75\best_meta_model


In [19]:
#no_test
from nbdev.export import notebook2script
notebook2script();

Converted .~75-meta-model.ipynb.
Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 54-utility-functions.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 65-all-annotations-model-training.ipynb.
Converted 66-es-guatemala-model-training.ipynb.
Converted 67-death-and-birth-records-together.ipynb.
Converted 70-exhaustive-training.ipynb.
Converted 71-relationship-builder.ipynb.
Converted 72-full-volume-processor.ipynb.
Converted 73-table-output.ipynb.
Converted 74-validation-visuals.ipynb.
Converted 75-meta-model.ipynb.
Converted 76-val-check-verif.ipynb.
